In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from __future__ import division
import copy
from scipy.stats import chi2_contingency
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
from sklearn import tree
#from numpy.ndarray import *
import graphviz 
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
import datetime as dt
import string
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


C:\Users\frank\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Read data

In [2]:
complaint = pd.read_csv('E:/New folder/Consumer_Complaints.csv')
test = pd.read_csv('E:/New folder/test_set.csv', header = None, names= ['ID', 'Complaint ID'])

In [3]:
df = complaint.copy()
df.shape #check each variable's type


(837344, 18)

# Preprocess Data



## 1. Missing data manipulation

In [4]:
#checking missing values
[(np.sum(df.iloc[:,i].isnull())/df.shape[0],df.columns[i])
 for i in range(df.shape[1])]

[(0.0, 'Date received'),
 (0.0, 'Product'),
 (0.28082962318951349, 'Sub-product'),
 (0.0, 'Issue'),
 (0.55746980930179235, 'Sub-issue'),
 (0.78918819505484006, 'Consumer complaint narrative'),
 (0.72776660488401423, 'Company public response'),
 (0.0, 'Company'),
 (0.011956854052814613, 'State'),
 (0.012002235640310315, 'ZIP code'),
 (0.85991539916688964, 'Tags'),
 (0.60855753429892612, 'Consumer consent provided?'),
 (0.0, 'Submitted via'),
 (0.0, 'Date sent to company'),
 (0.0, 'Company response to consumer'),
 (0.0, 'Timely response?'),
 (0.082184860702411433, 'Consumer disputed?'),
 (0.0, 'Complaint ID')]

In [190]:
df2 = df.copy()
df2.shape

(837344, 18)

In [191]:
#Variables with more than 50% missing variables will be deleted. Therefore, Sub-issue, 
#Consumer complaint narrative, Company public response, Tags, and Consumer consent provided? Will be removed. 
#Complain ID will be removed because it does not significant in this study.

df2.drop(['Sub-issue', 'Consumer complaint narrative','Company public response','Tags','Consumer consent provided?'], axis=1, inplace=True)

In [192]:
#"consumer consent provided", "company response to consumer", "consumer disputed" happend after consumer responds 
#and are removed from analysis
df2.drop(['Company response to consumer', 'Consumer disputed?'], axis=1, inplace=True)

In [193]:
#Observations missing State, ZIP code Are removed.
df2 = df2 [ -pd.isnull(df2.loc[:,'State'])] #remove missing obs in  State?
df2 = df2 [ -pd.isnull(df2.loc[:,'ZIP code'])] #remove missing obs in ZIP code

## 2. Rename variables

In [194]:
#rename df2
df2 =  df2.rename(index=str, columns={"Date received": "Date_re", "ZIP code": "Zip", "Submitted via": "Submitted", 
                               "Date sent to company": "Date_sent", "Timely response?": "Timely_response", "Sub-product":'Subproduct'})

## 3. Convert variables

convert date to weekday

In [195]:
date_re = pd.Series([dt.date.weekday(dt.datetime.strptime(i, "%m/%d/%Y").date()) for i in df2.Date_re])
date_sent = pd.Series([dt.date.weekday(dt.datetime.strptime(i, "%m/%d/%Y").date()) for i in df2.Date_sent])

df2.Date_re = date_re.values
df2.Date_sent = date_sent.values

In [196]:
df2.head()

,Date_re,Product,Subproduct,Issue,Company,State,Zip,Submitted,Date_sent,Timely_response,Complaint ID
0,2,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",M&T BANK CORPORATION,MI,48382,Referral,0,Yes,759217
1,5,Credit reporting,NaN,Incorrect information on credit report,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,Web,2,Yes,2141773
2,0,Consumer Loan,Vehicle loan,Managing the loan or lease,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Web,3,Yes,2163100
3,6,Credit card,NaN,Bankruptcy,AMERICAN EXPRESS COMPANY,ID,83854,Web,1,Yes,885638
4,5,Debt collection,Credit card,Communication tactics,"CITIBANK, N.A.",VA,23233,Web,5,Yes,1027760


In [197]:
diff = df2.loc[:,'Date_re'] - df2.loc[:,'Date_sent']
print ('crosstab dependent var vs. date diff')
pd.crosstab(diff, df2.Timely_response)

crosstab dependent var vs. date diff


Timely_response,No,Yes
row_0,,
-6,25,1004
-5,53,740
-4,382,7718
-3,1160,27972
-2,1949,59123
-1,2695,99630
0,11874,434710
1,1443,42986
2,1499,46105


In [198]:
#calculate date diff
df2.Date_diff = diff
df2 = df2.drop(['Date_re','Date_sent'],axis=1)

In [199]:
print ('crosstab dependent var vs. product')
pd.crosstab(df2.Product, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.Product, df2.Timely_response).iloc[:,1]

crosstab dependent var vs. product


Product
Bank account or service                                                          -5875
Checking or savings account                                                      -2596
Consumer Loan                                                                    13358
Credit card                                                                     -53371
Credit card or prepaid card                                                      -4821
Credit reporting                                                               -129183
Credit reporting, credit repair services, or other personal consumer reports     -1111
Debt collection                                                                 250988
Money transfer, virtual currency, or money service                                -466
Money transfers                                                                     -6
Mortgage                                                                        -58689
Other financial service            

Recode product, categorized depending on their frequency

In [200]:
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Consumer Loan')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Debt collection')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Other financial service')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Payday loan')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Vehicle loan or lease')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Student loan')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Other financial service')), 'Product'] = 'LOAN'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Virtual currency')), 'Product'] = 'LOAN'


df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Bank account or service')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Checking or savings account')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Credit card')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Credit reporting')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Money transfer')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Money transfers')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Mortgage')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Prepaid card')), 'Product'] = 'Other'
df2.loc[(df2['Product'].notnull()) & (df2['Product'].str.contains('Prepaid card')), 'Product'] = 'Other'

Recode state, based on their response

In [168]:
print ('crosstab dependent var vs. State')
#and   https://stackoverflow.com/questions/21415661/logic-operator-for-boolean-indexing-in-pandas 
#not   https://stackoverflow.com/questions/15998188/how-can-i-obtain-the-element-wise-logical-not-of-a-pandas-series
#or    https://stackoverflow.com/questions/24775648/element-wise-logical-or-in-pandas

#flatten array  .flatten, .ravel()
result = pd.crosstab(df2.State, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.State, df2.Timely_response).iloc[:,1]
state1 = result [result >100].index.values
#state2 = result [(result [result >-100])  & (result[result<=100])].index.values
state2 = np.intersect1d((result [result >-100]).index.values,(result[result<=100]).index.values)
state3 = result [result <=-100].index.values
state = np.concatenate((state1,state2,state3))
region = ['Neg'] * state1.shape[0] + ['Med'] * state2.shape[0] + ['Pos'] * state3.shape[0]

df2.State = df2.State.apply(lambda x: dict(zip(state, region))[x])


crosstab dependent var vs. State


In [224]:
#len([i for i in list(state1, state2, state3)])
#len(region), len(state),len(df2.State.unique()),len(state1),len(state2),len(state3),state[state1.isin(list(state))]
#[i for i in df2.State.unique() if i not in state],result.loc[state2]

#np.intersect1d((result [result >-100]).index.values,(result[result<=100]).index.values)
#result[(result>-100)&(result<=100)].index.values,result [result >-100].index.values, result[result<=100].index.values,np.intersect1d((result [result >-100]).index.values,(result[result<=100]).index.values)
#result[(result [result >-100])&(result[result<=100])].index.values, result [result >-100].index.values, result[result<=100].index.values,np.intersect1d((result [result >-100]).index.values,(result[result<=100]).index.values)
#result = pd.crosstab(df2.State, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.State, df2.Timely_response).iloc[:,1]
#result[result<=100]&result[result>-100]
pd.concat([pd.Series((result<=100)&(result>-100)),pd.Series(result[result<=100]&result[result>-100])],axis=1)
result[result<=100].head(),result[result>-100].head()
#(result[result<=100])

(State
 AA    -15
 AE    -54
 AP     81
 AR   -242
 AS    -25
 dtype: int64, State
 AA     -15
 AE     -54
 AK     284
 AL    2106
 AP      81
 dtype: int64)

Recode subproduct based on their frequency

In [262]:
print ('Check nan\'s value')
df2.loc[df2.Subproduct.isnull(),'Timely_response'].value_counts()

Check nan's value


Yes    232068
No       1876
Name: Timely_response, dtype: int64

In [274]:
print ('crosstab dependent var vs. Subproduct')
result = (pd.crosstab(df2.Subproduct, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.Subproduct, df2.Timely_response).iloc[:,1])/df2.Subproduct.value_counts()

df2.loc[df2.Subproduct.isnull(),'Subproduct'] = 'Pos'

sub_neg = result [result >0.1].index.values
sub_med = np.intersect1d((result [result >-0.1]).index.values,(result[result<=0.1]).index.values)
sub_pos = result [result <=-0.1].index.values

sub = np.concatenate((sub_neg,sub_med,sub_pos,['Pos']))
cat = ['Neg'] * sub_neg.shape[0] + ['Med'] * sub_med.shape[0] + ['Pos'] * sub_pos.shape[0] + ['Pos']
df2.Subproduct = df2.Subproduct.apply(lambda x: dict(zip(sub, cat))[x])
pd.crosstab(df2.Subproduct,df2.Timely_response)

crosstab dependent var vs. Subproduct


Timely_response,No,Yes
Subproduct,,
Med,1161,41175
Neg,13247,186669
Pos,8807,576234


In [28]:
#upper characters in variable

#loan = ['LOAN', 'MORTGAGE','DEBT']
#df2.loc[df2['Subproduct'].notnull(), 'Subproduct'] = pd.Series([x.upper() 
#for x in df.loc[df['Sub-product'].notnull(),'Sub-product']])  # convert to uppercase
#len(set([[x in y.split()] for y in df2.Subproduct]).intersection(set(loan))) != 0


C:\Users\frank\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [273]:

(pd.crosstab(df2.Issue, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.Issue, df2.Timely_response).iloc[:,1])/df2.Issue.value_counts()

APR or interest rate                                            -0.683748
Account opening, closing, or management                         -0.018860
Account terms and changes                                       -0.025000
Adding money                                                    -0.818182
Advertising                                                     -1.000000
Advertising and marketing                                       -0.682173
Advertising and marketing, including promotional offers         -0.918182
Advertising, marketing or disclosures                           -1.000000
Application processing delay                                    -0.664179
Application, originator, mortgage broker                         0.283014
Applied for loan/did not receive money                           1.315789
Applying for a mortgage                                         -0.384615
Applying for a mortgage or refinancing an existing mortgage      0.221239
Arbitration                           

In [285]:
print ('crosstab Issue vs Timely_response')
result = (pd.crosstab(df2.Issue, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.Issue, df2.Timely_response).iloc[:,1])/df2.Issue.value_counts()
sub_neg = result [result >0.1].index.values
sub_med = np.intersect1d((result [result >-0.1]).index.values,(result[result<=0.1]).index.values)
sub_pos = result [result <=-0.1].index.values

sub = np.concatenate((sub_neg,sub_med,sub_pos))
cat = ['Neg'] * sub_neg.shape[0] + ['Med'] * sub_med.shape[0] + ['Pos'] * sub_pos.shape[0] 
df2.Issue = df2.Issue.apply(lambda x: dict(zip(sub, cat))[x])
pd.crosstab(df2.Issue,df2.Timely_response)

crosstab Issue vs Timely_response


Timely_response,No,Yes
Issue,,
Med,2033,72701
Neg,14517,217045
Pos,6665,514332


Recode Issue, issues contain (loan, mortgage, debt) is recoded as LOAN. issues contain credit is recoded as credit, others are recoded as other

In [29]:
#recode variable based on their frequency

#Product_freq = df2.Issue.value_counts()/(df2.shape[0])
#p_cat = list(Product_freq[Product_freq < 0.1].index) #change index to array
#df2.loc[:,'Issue'][ df2.loc[:,'Issue'].isin( p_cat)] = 'Other'

C:\Users\frank\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
df2.head()

,Date_re,Product,Subproduct,Issue,Company,State,Zip,Submitted,Date_sent,Timely_response,Complaint ID
0,2,Mortgage,LOAN,LOAN,M&T BANK CORPORATION,NE,48382,Referral,0,Yes,759217
1,5,Credit reporting,NaN,Credit,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",SE,352XX,Web,2,Yes,2141773
2,0,Consumer Loan,LOAN,LOAN,"CITIZENS FINANCIAL GROUP, INC.",NE,177XX,Web,3,Yes,2163100
3,6,Credit card,NaN,Other,AMERICAN EXPRESS COMPANY,NW,83854,Web,1,Yes,885638
4,5,Debt collection,BANKING_SERVICE,Other,"CITIBANK, N.A.",SE,23233,Web,5,Yes,1027760


Check dependent variable frequency

In [31]:
df2.Timely_response.value_counts()/(df2.shape[0])

Yes    0.971939
No     0.028061
Name: Timely_response, dtype: float64

Recode company, convert cat to continuous based on frequency

In [32]:
#n = df2.Company.value_counts()/(df2.shape[0])
#dic = dict(zip(n.index,n.values/n[0]))
#df2['Company'] = df2['Company'].apply(lambda x: dic[x])

In [ ]:
print ('crosstab Company vs Timely_response')
result = (pd.crosstab(df2.Company, df2.Timely_response).iloc[:,0] * 35 - pd.crosstab(df2.Company, df2.Timely_response).iloc[:,1])/df2.Company.value_counts()
sub_neg = result [result >0.1].index.values
sub_med = np.intersect1d((result [result >-0.1]).index.values,(result[result<=0.1]).index.values)
sub_pos = result [result <=-0.1].index.values

sub = np.concatenate((sub_neg,sub_med,sub_pos))
cat = ['Neg'] * sub_neg.shape[0] + ['Med'] * sub_med.shape[0] + ['Pos'] * sub_pos.shape[0] 
df2.Company = df2.Company.apply(lambda x: dict(zip(sub, cat))[x])
pd.crosstab(df2.Company,df2.Timely_response)

crosstab Company vs Timely_response



ZIP: convert cat to continuous based on frequency

In [33]:
n = df2.Zip.value_counts()/(df2.shape[0])
dic = dict(zip(n.index,n.values/max(n)))
df2['Zip'] = df2['Zip'].apply(lambda x: dic[x])

## 3. Encoding to change categorical to factor

In [36]:
df3 = df2.copy()
lb_make = LabelEncoder()
for i in ['Product', 'Subproduct','Issue','State','Submitted','Timely_response','Date_re','Date_sent','Company']:
    df3[i+"_code"] = lb_make.fit_transform(df3[i])

Remove first 10 columns and remain one hot encoding columns

In [37]:
df3 = df3.loc [:,['Date_re_code','Date_sent_code','Company_code','Zip','Product_code', 'Subproduct_code','Issue_code','State_code',
            'Submitted_code','Timely_response_code','Complaint ID']]

In [38]:
#chi-squre test

[chi2_contingency(pd.crosstab(np.asarray(df3.iloc[:,i]), np.asarray(df3.loc[:,'Timely_response_code'])))[:2] for i in range(df3.shape[1])]



[(55.981517660971917, 2.9361179005562298e-10),
 (408.67969586922345, 3.8041565897499607e-85),
 (243914.44198453895, 0.0),
 (1189.7438363263971, 4.8973434419626942e-60),
 (16523.256552289797, 0.0),
 (12223.727104451322, 0.0),
 (3869.2800789372732, 0.0),
 (198.79523215196596, 6.821615307918483e-42),
 (609.33105664345874, 1.9483970375036275e-129),
 (827256.33539423801, 0.0),
 (827292.99999999965, 0.49948309090780257)]

In [39]:
df3.head()

,Date_re_code,Date_sent_code,Company_code,Zip,Product_code,Subproduct_code,Issue_code,State_code,Submitted_code,Timely_response_code,Complaint ID
0,2,0,2420,0.253326,6,2,1,0,4,1,759217
1,5,2,3864,0.143090,3,0,0,3,5,1,2141773
2,0,3,756,0.009775,1,2,1,0,5,1,2163100
3,6,1,108,0.013304,2,0,2,1,5,1,885638
4,5,5,755,0.028781,5,1,2,3,5,1,1027760


In [ ]:
pd.crosstab(df3.Date_re_code, Timely_response_code)

# Model

## 1. Tree model

In [110]:


#X_train = df3.ix[:int(0.9*df3.shape[0]),df3.columns != 'Timely_response_code']
#y_train = df3.ix[:int(0.9* df3.shape[0]),'Timely_response_code']

#X_test = df3.ix[int(0.9*df3.shape[0]):,df3.columns != 'Timely_response_code']
#y_test = df3.ix[int(0.9*df3.shape[0]): ,'Timely_response_code']
index = df3.loc[:,'Complaint ID'].isin(test.loc[:,'Complaint ID'])
X_train = df3.loc[~index,df3.columns !='Timely_response_code']
X_test = df3.loc[index,df3.columns !='Timely_response_code']

y_train = df3.loc[~index,'Timely_response_code']
y_test = df3.loc[index,'Timely_response_code']

X_train = X_train.drop(['Complaint ID'],axis=1)
X_test = X_test.drop(['Complaint ID'],axis=1)


In [111]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                                min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [115]:
sklearn.metrics.roc_auc_score(y_test,clf_gini.predict(X_test))

0.5

In [113]:
X_test.head(),X_train.head()

(    Date_re_code  Date_sent_code  Company_code       Zip  Product_code  \
 7              0               0          1477  0.083627             3   
 11             4               3          3790  0.013033             8   
 19             2               2          1477  0.015205             3   
 23             2               2          2739  0.019278             6   
 26             3               3          1927  0.010589             6   
 
     Subproduct_code  Issue_code  State_code  Submitted_code  
 7                 0           0           3               5  
 11                2           1           0               5  
 19                0           0           4               2  
 23                2           1           1               5  
 26                2           1           0               5  ,
    Date_re_code  Date_sent_code  Company_code       Zip  Product_code  \
 0             2               0          2420  0.253326             6   
 1             5     

In [114]:
result_tree = np.mean(cross_val_score(clf_gini, X, y, cv=10, scoring="roc_auc"))
Print (result_tree)

KeyboardInterrupt: 

The AUC is 0.80769115892815968

In [146]:
clf.feature_importances_

array([ 0.        ,  0.        ,  0.99007413,  0.        ,  0.00992587,
        0.        ,  0.        ,  0.        ,  0.        ])

The result implies the most important feature is company.

## 2. Random forest

In [165]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
result_rf = np.mean(cross_val_score(clf, X, y, cv=10, scoring="roc_auc"))

The AUC is 0.75434656690566382

In [ ]:
clf_svm = svm.SVR()
clf_svm.fit(X_train, y_train)
sklearn.metrics.roc_auc_score(y_test,clf_svm.predict(X_test))

Summary: the most important variable that influences Timely response is company. AUC of random forest and tree is 0.75 and 0.8 respectively